# Zadanie bonusowe

Nie robię EDA, zajmę się tylko dopasowaniem regresji i wpływem metod regularyzacji LASSO i RIDGE na wynik.

### Opis metod. 
W przypadku regularyzacji do minimalizacji funkcji kryterialnej dodawana jest kara zależna od wartości parametrów strukturalnych w modelu regresji. LASSO i RIDGE są pomocne w przypadku gdy występuje współliniowość.

Lasso stosuje karę L1, to jest sumę wartości bezwzględnych, RIDGE karę L2, to jest suma kwadrató. Dodatkowa kara może mieć różny wpływ w zależności od hiperparametru który wyznacza wagę z którą kara będzie wchodziła do sumy ze tandardową funkcją kryterialną.

Estymatory LASSO i RIDGE są wyznaczane jednoznacznie i istnieją, jeżeli wejściowa ramka danych ma pełen rząd. RIDGE ma analityczną postać, natomiast estymatory LASSO należy przybliżać numerycznie.

Lasso jest przykładem miękkiego i twardego obcinania, RIDGE przykładme tylko miękkiego obcinania. LASSO automatycznie dokonuje 
selekcji zmiennych, natomiast jest ona zależna od hiperparametru. Są pewne podpowiedzi jak ten parametr stroić, na przykład za pomocą metody kroswalidacji 1se.

RIDGE w orginalnym artykule została zaproponowana jako metoda regularyzacji i poprawienia numerycznych własności przy obliczaniu parametrów, zwiększała ona wartości na diagonali macierzy X'X, gdzie X to wejściowa rama danych. 

In [1]:
import pandas as pd
import sklearn
import numpy as np

In [2]:
df=pd.read_csv("C:\\Users\\huber\\Dropbox\\PW\\2 SEMESTR\\Wstep do uczenia maszynowego\\allegro-api-transactions.csv")

In [3]:
df.head()

,lp,date,item_id,categories,pay_option_on_delivery,pay_option_transfer,seller,price,it_is_allegro_standard,it_quantity,it_is_brand_zone,it_seller_rating,it_location,main_category
0,0,2016-04-03 21:21:08,4753602474,"['Komputery', 'Dyski i napędy', 'Nośniki', 'No...",1,1,radzioch666,59.99,1,997,0,50177,Warszawa,Komputery
1,1,2016-04-03 15:35:26,4773181874,"['Odzież, Obuwie, Dodatki', 'Bielizna damska',...",1,1,InwestycjeNET,4.90,1,9288,0,12428,Warszawa,"Odzież, Obuwie, Dodatki"
2,2,2016-04-03 14:14:31,4781627074,"['Dom i Ogród', 'Budownictwo i Akcesoria', 'Śc...",1,1,otostyl_com,109.90,1,895,0,7389,Leszno,Dom i Ogród
3,3,2016-04-03 19:55:44,4783971474,"['Książki i Komiksy', 'Poradniki i albumy', 'Z...",1,1,Matfel1,18.50,0,971,0,15006,Wola Krzysztoporska,Książki i Komiksy
4,4,2016-04-03 18:05:54,4787908274,"['Odzież, Obuwie, Dodatki', 'Ślub i wesele', '...",1,1,PPHU_RICO,19.90,1,950,0,32975,BIAŁYSTOK,"Odzież, Obuwie, Dodatki"


In [4]:
df1=df.drop(columns=["date", "item_id", "seller", "categories", "lp"])
df1['it_location'] = df1['it_location'].str.lower()

#### ZASTOSOWAC TARGET ENCODING I WYJASNIC CZEMU TAKI A NIE INNY PARAMETR SMOOTHING
"Smoothing effect to balance categorical average vs prior. Higher value means stronger regularization. The value must be strictly bigger than 0". Trzeba by poczytać artykuły testujące wpływ lambdy i sugerujące wybór tej lambdy, parametru wysmuklającego, regularyzującego. W TargetEncoder chodzi o ważenie średniej globalnej i średniej dla grup. Im jest on wyższy tym mniej Target Encoder będzie dopasowany do wartości zmiennej celu. Chcemy się bronić, przed zbytnim dopasowaniem encodera do danych, używania informacji o wektorze odpowiedzi.  All in all, zależy nam na wariancji, więc dobieram ten parametr jako 20 dla main_category, a dla bardzo róznorodnej it_location parametr jako bardzo wysoki. 

In [5]:
import pandas as pd
from category_encoders import TargetEncoder
encoder = TargetEncoder(smoothing=20)
df1['main_encoded'] = encoder.fit_transform(df['main_category'], df['price'])
df1['main_encoded']
df1=df1.drop(columns="main_category")


encoder = TargetEncoder(smoothing=200)
df1['location_encoded'] = encoder.fit_transform(df['it_location'], df['price'])
df1['location_encoded']
df1=df1.drop(columns="it_location")

C:\Users\huber\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\huber\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [6]:
df1

,pay_option_on_delivery,pay_option_transfer,price,it_is_allegro_standard,it_quantity,it_is_brand_zone,it_seller_rating,main_encoded,location_encoded
0,1,1,59.99,1,997,0,50177,121.810064,85.423398
1,1,1,4.90,1,9288,0,12428,75.858049,85.423398
2,1,1,109.90,1,895,0,7389,72.434754,62.077652
3,1,1,18.50,0,971,0,15006,25.027497,50.909648
4,1,1,19.90,1,950,0,32975,75.858049,117.182219
...,...,...,...,...,...,...,...,...,...
420015,0,0,180.00,0,0,0,176,107.534272,35.863907
420016,1,1,14.99,1,2,0,34851,28.128577,66.800385
420017,1,1,5.99,1,470,0,983,75.858049,46.005308
420018,1,0,200.00,0,0,0,163,71.206386,106.203076


In [7]:
# MODEL LINIOWY I CENA PRODUKTU 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
x=df1.loc[list(range(10000)),["pay_option_on_delivery", "pay_option_transfer", "it_is_allegro_standard", "it_is_brand_zone", "it_seller_rating","location_encoded" ,"main_encoded"]]
y=df1.loc[list(range(10000)),"price"]
LR=LinearRegression()
LASSO=linear_model.Lasso()
RIDGE=Ridge()
ELASTIC=ElasticNet()
SVM=SVR()
#alpha=np.logspace(-6, 6, 13)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.2)

In [9]:
X_train

,pay_option_on_delivery,pay_option_transfer,it_is_allegro_standard,it_is_brand_zone,it_seller_rating,location_encoded,main_encoded
2302,1,1,1,0,69769,46.614012,134.426801
6738,1,1,0,0,17027,477.523042,134.426801
1143,1,1,1,0,20455,4.407974,72.434754
6697,1,1,1,0,14118,66.800385,72.434754
5862,1,1,1,0,766,79.069454,72.434754
...,...,...,...,...,...,...,...
9559,1,1,1,0,9599,106.203076,75.858049
9442,1,1,1,0,8737,75.730710,72.434754
3616,1,1,1,0,54947,45.892407,25.027497
3388,1,1,1,0,46279,4.407974,72.434754


In [15]:
y_train

335983      5.00
411822     22.99
130521     28.00
203586     59.90
366672      1.99
           ...  
34777     199.99
154914     22.89
188631     20.00
251788    377.90
161394     29.99
Name: price, Length: 336016, dtype: float64

Zostało wykonane strojenie hiperparametrów

In [23]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
params={'alpha':[100,500,1000,2000],
'l1_ratio' :np.arange(0, 1, 0.1)
       }
RSELAS=GridSearchCV(ELASTIC, params, scoring='neg_mean_absolute_error', cv=3, n_jobs=-1)
RSELAS.fit(X_train, y_train)

params={'alpha':[100,5000,20000 ,30000, 60000,100000,100000000 ,100000000000000]}
RSLASSO=GridSearchCV(LASSO, params, scoring='neg_mean_absolute_error', cv=3, n_jobs=-1)
RSLASSO.fit(X_train, y_train)

RSRIDGE=GridSearchCV(RIDGE, params, scoring='neg_mean_absolute_error', cv=3, n_jobs=-1)
RSRIDGE.fit(X_train, y_train)
params={'C':[10,100,500,1000],
        'kernel':['rbf'],
        'gamma':[0.01,0.001,0.0001],
       }
RSSVR=RandomizedSearchCV(SVM, params, scoring='neg_mean_absolute_error', cv=3, n_jobs=-1)
RSSVR.fit(X_train, y_train)

print('ELASTIC NET PRAMETERS {}'.format(RSELAS.best_params_) )
print('LASSO PARAMETERS {}'.format(RSLASSO.best_params_) )
print('RIDGE PARAMETERS {}'.format(RSRIDGE.best_params_) )
print('SVM PARAMETERS {}'.format(RSSVR.best_params_) )

ELASTIC NET PRAMETERS {'alpha': 500, 'l1_ratio': 0.0}
LASSO PARAMETERS {'alpha': 5000}
RIDGE PARAMETERS {'alpha': 100000000}
SVM PARAMETERS {'kernel': 'rbf', 'gamma': 0.01, 'C': 100}


WYNIKI DLA POSZCZEGÓLNYCH REGRESJI

In [29]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

LASSO=linear_model.Lasso(alpha=5000)
RIDGE=Ridge(alpha=100000000)
SVM=SVR(kernel='rbf', gamma= 0.01, C=100)

LR.fit(X_train, y_train)
LASSO.fit(X_train, y_train)
RIDGE.fit(X_train, y_train)
SVM.fit(X_train, y_train)

y_pred=LR.predict(X_test)
print("Normal Regression MRSE", mean_squared_error(y_test, y_pred, squared=False))
print("Normal Regression R2 in Mincern Zarnovitz regression", r2_score(y_test, y_pred))
y_pred=LASSO.predict(X_test)
print("LASSO MRSE", mean_squared_error(y_test, y_pred, squared=False))
print("LASSO R2 in Mincern Zarnovitz regression", r2_score(y_test, y_pred))

RID=RIDGE.fit(X_train,y_train)
y_pred=RID.predict(X_test)
print("RIDGE MRSE", mean_squared_error(y_test, y_pred, squared=False))
print("RIDGE R2 in Mincern Zarnovitz regression", r2_score(y_test, y_pred))

#y_pred=ELASTIC().predict(X_test)
#print("ELASTIC NET MRSE", mean_squared_error(y_test, y_pred, squared=False))
#print("ELASTIC NET R2 in Mincern Zarnovitz regression", r2_score(y_test, y_pred))

y_pred=SVM.predict(X_test)
print("SVR MRSE", mean_squared_error(y_test, y_pred, squared=False))
print("MRSE R2 in Mincern Zarnovitz regression", r2_score(y_test, y_pred))

Normal Regression MRSE 189.54273745670474
Normal Regression R2 in Mincern Zarnovitz regression 0.2952856781274431
LASSO MRSE 219.41971655686936
LASSO R2 in Mincern Zarnovitz regression 0.05561278298676886
RIDGE MRSE 215.09626549678254
RIDGE R2 in Mincern Zarnovitz regression 0.0924625802464708
SVR MRSE 221.15385823533123
MRSE R2 in Mincern Zarnovitz regression 0.04062622764572177


Wnioski:

Elastic Net okazał się być RIDGE podczas optymalizacji hiperparametrów, więc został wyrzucony z analizy. Co ciekawe normalna regresja bez regularyzacji w tym przypadku wypadła najkorzystniej. Może być to po prostu wpływ specyfiki dobranych zmiennych i wykorzystanie target encodingu. Słabo radził sobie także SVR. 

Niemniej jednak regularyzacja jest potężnym konceptem. Chociażby dlatego, że pozwala walczyć ze współliniowością, jest szybkim algorytmem dobierającym zmienne w przypadku Lasso, sprawia, że konkretne algorytmy są bardziej elastyczne w kontekście modelowania zjawisk dla różnych typów danych, poprawia własności numeryczne niektórych algorytmów. 